In [1]:
import numpy as np


import numba as nb
from numba import jit

In [2]:
@jit(nopython=True)
def func(x):
    return np.exp(-(1000.*(x[0]-0.8)**2+500.*(x[1]-0.2)**2 +1800.*(x[2]-0.3)**2))  


In [3]:
#this is the compiled Metropolis
#The only thing that needed to be changed is the function min (jit complained).

@jit(nopython=True)
def compiled_Metropolis(Target,xin,N=10000,eps=0.01,burn_in=0,thinning=10,_fixed_interval=True):
    '''
    burn_in (integer): do not store the first 'burn_in' accepted steps (the first few steps could be in a region with low probability).
    thinning (integer): store 1 every 'thinning' accepted steps (makes the sample more independent).
    _fixed_interval (boolean): if True keps x in [0,1].
    '''
    Dim=len(xin)
    accepted=[]

    _n_b=0
    _n_t=0

    x=xin[:]
    
    func_x=Target(x)

    for i in np.arange(N):
        
        can=[]
        for d in range(Dim):
            while True:
                #step=(np.random.rand()*2-1)*eps[d]
                step=np.random.normal(0,eps[d])
                
                if _fixed_interval:
                    if x[d]+step<1 and x[d]+step>0:#check if x stays inside the cube [0,1]^dim (I want the point in here for integration)
                        break
                else:
                    break

            can.append(x[d] + step) #candidate

        func_can=Target(can)


        if func_x<1e-50:
            a=1
        else:    
            if 1<func_can/func_x:
                a=i
            else:    
                a = func_can/func_x #acceptance probability for symmetric prior
        
        u = np.random.rand()

        if u < a :
            x = can[:]
            _n_b+=1
            _n_t+=1
            if _n_b>=burn_in and _n_t>thinning:

                _n_t=0
                accepted.append(x)#accept sample


        func_x=Target(can)
    return np.array(accepted)

In [4]:
#this is the pure python Metropolis
def Metropolis(Target,xin,N=10000,eps=0.01,burn_in=0,thinning=10,_fixed_interval=True):
    '''
    burn_in (integer): do not store the first 'burn_in' accepted steps (the first few steps could be in a region with low probability).
    thinning (integer): store 1 every 'thinning' accepted steps (makes the sample more independent).
    _fixed_interval (boolean): if True keps x in [0,1].
    '''
    Dim=len(xin)
    accepted=[]

    _n_b=0
    _n_t=0

    x=xin[:]
    
    func_x=Target(x)

    for i in np.arange(N):
        
        can=[]
        for d in range(Dim):
            while True:
                #step=(np.random.rand()*2-1)*eps[d]
                step=np.random.normal(0,eps[d])
                
                if _fixed_interval:
                    if x[d]+step<1 and x[d]+step>0:#check if x stays inside the cube [0,1]^dim (I want the point in here for integration)
                        break
                else:
                    break

            can.append(x[d] + step) #candidate

        func_can=Target(can)


        if func_x<1e-50:
            a=1
        else:    
            if 1<func_can/func_x:
                a=i
            else:    
                a = func_can/func_x #acceptance probability for symmetric prior
        
        u = np.random.rand()

        if u < a :
            x = can[:]
            _n_b+=1
            _n_t+=1
            if _n_b>=burn_in and _n_t>thinning:

                _n_t=0
                accepted.append(x)#accept sample


        func_x=Target(can)
    return np.array(accepted)

In [8]:
%%timeit -n 2 -r 2
_=Metropolis(Target=func,xin=[0.5,0.1,0.7],N=200000,eps=[0.01,0.04,0.02],burn_in=50000,thinning=80)

11 s ± 42.9 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


In [11]:
%%timeit -n 2 -r 2
_=compiled_Metropolis(Target=func,xin=[0.5,0.1,0.7],N=200000,eps=[0.01,0.04,0.02],burn_in=50000,thinning=80)

104 ms ± 3.02 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
